<a href="https://colab.research.google.com/github/jamahalwin/chicago-crime-analysis/blob/master/ML_Chicago_Crime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Chicago Crime Analysis

In [0]:
# Use seaborn for pairplot
!pip install seaborn

# Use some functions from tensorflow_docs
!pip install git+https://github.com/tensorflow/docs

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [0]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [0]:
dataset_path =  keras.utils.get_file('chicago-crime-sample.data', 'https://raw.githubusercontent.com/jamahalwin/chicago-crime-analysis/master/Chicago_Crimes_Project_Data.csv')
dataset_path

In [8]:
column_names = ['ID','Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
                'Description', 'Location Description', 'Arrest',
                'Domestic', 'Beat', 'District', 'Ward', 'Community Area',
                'FBI Code', 'X Coordinate', 'Y Coordinate', 'Year', 'Updated On',
                'Latitude', 'Longitude', 'Location']
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=",", skipinitialspace=False)

dataset = raw_dataset.copy()
dataset.tail()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
77,10509764,HZ249689,5/3/2016 12:15,061XX S MORGAN ST,820,THEFT,$500 AND UNDER,APARTMENT,FALSE,FALSE,712,7,16,68,6,1170696,1864158,2016,5/10/2016 15:56,41.78272837,-87.64972028,"(41.782728365, -87.649720276)"
78,10509769,HZ251010,5/3/2016 20:30,014XX N AVERS AVE,320,ROBBERY,STRONGARM - NO WEAPON,SIDEWALK,FALSE,FALSE,2535,25,30,23,3,1150444,1909392,2016,5/10/2016 15:56,41.90727403,-87.72279189,"(41.907274031, -87.722791892)"
79,10509770,HZ250329,5/3/2016 9:00,048XX W EDDY ST,486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,FALSE,TRUE,1634,16,38,15,08B,1143575,1923186,2016,5/10/2016 15:56,41.94525754,-87.74767892,"(41.945257538, -87.74767892)"
80,10509783,HZ251242,5/3/2016 12:15,013XX E 72ND ST,460,BATTERY,SIMPLE,"SCHOOL, PUBLIC, BUILDING",FALSE,FALSE,324,3,5,43,08B,1186321,1857530,2016,5/10/2016 15:56,41.76418539,-87.5926442,"(41.764185391, -87.592644199)"
81,10509784,HZ251276,5/3/2016 20:00,062XX N NEWCASTLE AVE,810,THEFT,OVER $500,STREET,FALSE,FALSE,1611,16,41,10,6,1130189,1940933,2016,5/10/2016 15:56,41.99419755,-87.79647287,"(41.994197554, -87.796472871)"
